## Test Notebook for inserting time stamps not chronologically into a zarr file

In [26]:
import xarray as xr
import pandas as pd
import numpy as np
import zarr
import os
import shutil
from shutil import copyfile
import sys

First greate an empty zarr, which will be build upon

In [27]:
DATASET_PATH = "test.zarr"

In [28]:
def create_dataset(datetime):
    """Create a spatial, single time step dataset with two variables."""
    dims = ("time", "lat", "lon")
    time = [pd.to_datetime(datetime)]
    w = 4000
    h = 2000
    lon = np.linspace(0, 4, w)
    lat = np.linspace(50, 52, h)
    precipitation_var = xr.DataArray(np.random.rand(1, h, w), coords=(time, lat, lon), dims=("time", "lat", "lon"))
    temperature_var = xr.DataArray(np.random.rand(1, h, w), coords=(time, lat, lon), dims=("time", "lat", "lon"))
    ds = xr.Dataset({"precipitation": precipitation_var, "temperature": temperature_var})
    return ds
    

## Save a single time step dataset with default chunking

In [29]:
ds = create_dataset("2018-01-01")


In [15]:
ds.time.encoding

{}

In [16]:
ds.precipitation.encoding

{}

In [30]:
ds.to_zarr(DATASET_PATH, mode="w")
ds.close()

In [31]:
ds = xr.open_zarr(DATASET_PATH)

In [32]:
ds.time.encoding

{'chunks': (1,),
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'units': 'days since 2018-01-01 00:00:00',
 'calendar': 'proleptic_gregorian',
 'dtype': dtype('int64')}

In [33]:
ds.precipitation.encoding

{'chunks': (1, 250, 500),
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float64')}

In [34]:
ds.close()

## Inspect default chunking

In [35]:
root_group = zarr.open("test.zarr", mode='a')

This zarr will only contain time stamps for every other day:

In [36]:
for i in range(2, 10, 2):
    print(i)
    ds = create_dataset(f"2018-01-0{i}")
    for var_name, var_array in root_group.arrays():
        var = ds[var_name]
        if 'time' in var.dims:            
            time_axis = var.dims.index('time')
            var_array.append(var, axis=time_axis)

2
4
6
8


In [ ]:
ds.time.encoding{'chunks': (1,),
                 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
                 'filters': None,
                 'units': 'days since 2018-01-01 00:00:00',
                 'calendar': 'proleptic_gregorian',
                 'dtype': dtype('int64')}

In [37]:
ds = xr.open_zarr("test.zarr")


ValueError: unable to decode time units 'days since 2018-01-01 00:00:00' with calendar 'proleptic_gregorian'. Try opening your dataset with decode_times=False.

In [ ]:
ds.close()

Create another dataset which contains only one time stamp and will be appended to the first data set, stored in test.zarr

In [ ]:
DATASET_SINGLE_PATH = "test_single.zarr"

In [ ]:
ds = create_dataset("2018-01-01")
ds.to_zarr(DATASET_SINGLE_PATH, mode="w")
ds.close()

In [ ]:
root_group = zarr.open("test_single.zarr", mode='a')

In [ ]:
# for i in range(4, 9, 2):
#     print(i)
#     ds = create_dataset(f"2018-01-0{i}")
#     for var_name, var_array in root_group.arrays():
#         var = ds[var_name]
#         if 'time' in var.dims:            
#             time_axis = var.dims.index('time')
#             var_array.append(var, axis=time_axis)

In [ ]:
ds = create_dataset(f"2018-01-04")
for var_name, var_array in root_group.arrays():
    var = ds[var_name]
    if 'time' in var.dims:            
        time_axis = var.dims.index('time')
        var_array.append(var, axis=time_axis)

In [ ]:
ds.close()

## -----------------------------------

In [ ]:
ds = xr.open_zarr(DATASET_PATH, decode_times=False)

In [ ]:
ds_single = xr.open_zarr(DATASET_SINGLE_PATH, decode_times=False)

In [ ]:
ds.close()
ds_single.close()

The first time stamp of both data sets is the same, in order to have the same time encoding. So in this case we are interested in the second time stamp of the ds_single

In [ ]:
def check_merge_or_append(src_path, dst_path):
    """Check whether the time stamp of the current input file is before or past the last time stamp of the created data cube"""
    ds_single = xr.open_zarr(src_path, decode_times=False)
    ds = xr.open_zarr(dst_path, decode_times=False)
    if (np.greater(ds_single.time.values, ds.time[-1].values)).all() == True:
        #append modus
        print("Append modus is chosen.")
    else:
        check_if_unique(src_path, dst_path)
        print("Merge modus is chosen.")

In [ ]:
def check_if_unique(src_path, dst_path):
    """Check if to be added time stamp is unique """
    ds_single = xr.open_zarr(src_path, decode_times=False)
    ds = xr.open_zarr(dst_path, decode_times=False)
    for src_idx in range(ds_single.time.shape[0]):
        mask = (np.greater(ds.time.values, ds_single.time[src_idx].values)==False)&(np.equal(ds.time.values, ds_single.time[src_idx].values)==True)
        if mask.all()==False:
            print("merging")
            merge_single_zarr_into_destination_zarr(src_path, dst_path, src_idx)
        else:
            print("All timestamps to be merged are aleady in destination data set, and are skipped.")
        

In [ ]:
def find_nearest_above(array, target):
    diff = array - target
    mask = np.ma.less_equal(diff, 0)
    # We need to mask the negative differences and zero
    # since we are looking for values above
    if np.all(mask):
        return None # returns None if target is greater than any value
    masked_diff = np.ma.masked_array(diff, mask)
    return masked_diff.argmin()

In [ ]:
def rename_directory(path_to_ds, old_index, new_time_i):
    ds = xr.open_zarr(path_to_ds, decode_times=False)
    for v in ds.variables:
        if (v != 'lat') and (v != 'lon'):
            path = os.path.join(path_to_ds, v)
            for root, dirs, files in os.walk(path):  
                for filename in files:
                    if (str(old_index)) in filename[0] and (v != "time"):
                        parts = filename.split('.',1)
                        new_name = (str(new_time_i) + '.{}').format(parts[1]) 
                        if new_name != path:
                             os.rename(os.path.join(path, filename), os.path.join(path, new_name))
                    elif (str(old_index)) in filename[0] and (v == "time"):
                        if str(new_time_i) != path:
                            os.rename(os.path.join(path, filename), os.path.join(path, str(new_time_i)))
                

In [ ]:
def adjust_zarray(dst_path, variable, line_to_adjust):
    with open((os.path.join(dst_path, variable, '.zarray')), 'r') as zarray:
        data = zarray.readlines()
    position = 8
    white_space = len(data[line_to_adjust])-1
    data[line_to_adjust] = (str(int(data[line_to_adjust][position])+1) + data[line_to_adjust][9:]).rjust(len(data[line_to_adjust][position])+white_space)

    with open((os.path.join(dst_path, variable, '.zarray')), 'w') as zarray:
        zarray.writelines(data)     

In [ ]:
def copy_into_target(src_path, dst_path, src_index):
    ds = xr.open_zarr(src_path, decode_times=False)
    for v in ds.variables:
        if (v != 'lat') and (v != 'lon'):
            path = os.path.join(src_path, v)
            for root, dirs, files in os.walk(path):  
                for filename in files:
                    if str(src_index) in filename[0]:
                        copyfile((os.path.join(src_path, v, filename)), (os.path.join(dst_path, v,  filename)))
            if v != "time":
                line_to_adjust = 18
            elif v == "time":
                line_to_adjust = 16
            adjust_zarray(dst_path, v, line_to_adjust)             

In [ ]:
def merge_single_zarr_into_destination_zarr(src_path, dst_path, old_idx):
    ds_single = xr.open_zarr(src_path, decode_times=False)
    ds = xr.open_zarr(dst_path, decode_times=False)
    new_idx = find_nearest_above(ds.time.values, ds_single.time[old_idx].values)
    ds.close()
    ds_single.close()
# Preparing the source directory with the single time stamp to be ready for merging 
# --> files of variables, excluding "lat" and "lon" need to be renamed
    rename_directory(src_path, old_idx, new_idx)
# Preparing the destinanation directory to be ready for single time stam to be merged 
# --> files of variables, excluding "lat" and "lon" need to be renamed
# The renaming needs to happen in reversed order and starting at the index of nearest above value:
    for i in reversed(range(new_idx,ds.time.shape[0])):
        rename_directory(dst_path, i, (i +1))
# Final step: copy the single time stamp files into the destination zarr and adjusting .zarray to the change. 
    copy_into_target(src_path, dst_path, new_idx)

In [ ]:
%%time
check_merge_or_append(DATASET_SINGLE_PATH,DATASET_PATH)

In [ ]:
# variable

In [ ]:
# with open((os.path.join(DATASET_PATH, "temperature", '.zarray')), 'r') as zarray:
#           data = zarray.readlines()
# data[18]

In [ ]:
# (str(int(data[18][8])+1) + data[18][9:]).rjust(len(data[18][8])+10)

In [ ]:
# with open((os.path.join(DATASET_PATH, "time", '.zarray')), 'r') as zarray:
#           data = zarray.readlines()
# data[16]

In [ ]:
# (data[16][len(data[line_to_adjust])-2])

In [ ]:
# len(data[16])-2

In [ ]:
# data[18][9:]


In [ ]:
# line_to_adjust = 16
# position = 8
# white_space = 10

In [ ]:
# (str(int(data[line_to_adjust][position])+1) + data[line_to_adjust][9:]).rjust(len(data[line_to_adjust][position])+white_space)

In [ ]:
# (str(int(data[line_to_adjust][position])+1) + data[line_to_adjust][9:]).rjust(len(data[line_to_adjust][position])+white_space)

In [ ]:
# len(data[line_to_adjust])

In [ ]:
# shutil.rmtree(DATASET_SINGLE_PATH)

In [ ]:
# shutil.rmtree(DATASET_PATH)